In [3]:
import numpy as np
import pandas as pd
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [4]:
df=pd.read_csv('traintoxic.csv')
dftest=pd.read_csv('testtoxic.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
dfnew = pd.read_csv('hatespeech_labels.csv')
dfnew.shape

(99799, 2)

In [6]:
tfidf=TfidfVectorizer(strip_accents='unicode', stop_words='english' )
x=df['comment_text']
x=tfidf.fit_transform(x)

In [7]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
for classn in class_names:
    y = df[classn]
    model = LogisticRegression(C=0.1, solver='sag')
    cv_score = np.mean(cross_val_score(model, x, y, cv=3, scoring='roc_auc'))
    print('CV score for class {} is {}'.format(classn, cv_score))

    model.fit(x, y)
#     with open(classn+'_model.pkl','wb') as f:
#         pickle.dump(model,f)

CV score for class toxic is 0.9556510256507112
CV score for class severe_toxic is 0.9833047360245181
CV score for class obscene is 0.9797383458961768
CV score for class threat is 0.9747058685108314
CV score for class insult is 0.9688366670158386
CV score for class identity_hate is 0.9671049834414474


In [8]:
import tweepy
import csv
import json

consumer_key = "oIbrARLL6QbmbrJJhdl6fh1Bn"
consumer_secret = "YMEgyrhiblOFhCmHE3pfco85MBrhs8mSF9lyOGI3GzAZXBVx2P"
access_key = "806491866992676865-kaU9QCjQgdQgLPgoDOe50wGdtddJ9xZ"
access_secret = "KWFUurS6omuhKstXV2aGjDk4tDgUHk0dZWs2uuZOqR5m3"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)


In [15]:
friend_ids=api.friends_ids(user_id='DhaarnaSethi')
for idx in friend_ids:
    fdata=api.user_timeline(id=idx)
    for tweets in fdata:
        txt=[tweets.text]
        txt=tfidf.transform(txt)
        for classn in class_names:
            y = df[classn]
            with open('./Models/'+classn+'_model.pkl','rb') as f:
                model=pickle.load(f)
                if model.predict(txt) ==1:
                    print(classn, "1")
pickle.dump(tfidf, open("vectorizer.pickle", "wb"))

toxic 1
obscene 1
insult 1
toxic 1
obscene 1
toxic 1
